In [2]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

torchvision 0.15.1+cu118 requires torch==2.0.0, but you'll have torch 1.13.0 which is incompatible.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you'll have torch 1.13.0 which is incompatible.


In [4]:
pip install transformers requests beautifulsoup4 pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

instantiate model


In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

c:\Users\chanc\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\chanc\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:02<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:02<?, ?B/s]

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [7]:
tokens=tokenizer.encode("It was a lovely day yesterday with my family and friends",return_tensors="pt")

In [12]:
tokens

tensor([[  101, 10197, 10140,   143, 83944, 11111, 68404, 10171, 11153, 11214,
         10110, 16119,   102]])

In [10]:
tokenizer.decode(tokens[0])

'[CLS] it was a lovely day yesterday with my family and friends [SEP]'

In [14]:
result=model(tokens)

In [16]:
result
result.logits

tensor([[-2.2078, -2.1744, -0.1616,  1.6057,  2.2278]],
       grad_fn=<AddmmBackward0>)

In [17]:
int(torch.argmax(result.logits))+1

5

collect reviews from yelp

In [19]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [20]:
reviews[0]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'

load reviews into dataframe and get sentiment score

In [22]:
import pandas as pd
import numpy as np
df=pd.DataFrame(np.array(reviews),columns=["review"])

In [23]:
df

,review
0,Great food amazing coffee and tea. Short walk ...
1,It was ok. Had coffee with my friends. I'm new...
2,Great staff and food. Must try is the pan fri...
3,Ricotta hot cakes! These were so yummy. I ate ...
4,I came to Social brew cafe for brunch while ex...
5,We came for brunch twice in our week-long visi...
6,It was ok. The coffee wasn't the best but it w...
7,I went here a little while ago- a beautiful mo...
8,Great coffee and vibe. That's all you need. C...
9,Great coffee and vibe. That's all you need. C...


In [24]:
def sentiment_score(review):
    tokens=tokenizer.encode(review,return_tensors="pt")
    result=model(tokens)
    return int(torch.argmax(result.logits))+1


In [25]:
sentiment_score(df['review'].iloc[1])

3

In [26]:
# we take 512 tokens because bert takes only 512 tokens at once 

df["sentiment_score"]=df['review'].apply(lambda x: sentiment_score(x[:512]))

In [27]:
df

,review,sentiment_score
0,Great food amazing coffee and tea. Short walk ...,5
1,It was ok. Had coffee with my friends. I'm new...,3
2,Great staff and food. Must try is the pan fri...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I came to Social brew cafe for brunch while ex...,5
5,We came for brunch twice in our week-long visi...,4
6,It was ok. The coffee wasn't the best but it w...,3
7,I went here a little while ago- a beautiful mo...,2
8,Great coffee and vibe. That's all you need. C...,5
9,Great coffee and vibe. That's all you need. C...,4
